## **Notebook 02-Splits (train/val/test + prompt_regression):**

**Objetivo**
- Crear un identificador estable doc_uid para unannotated (no tiene id).
- Generar particiones reproducibles: train/val/test y un conjunto pequeño prompt_regression.
- (En este NB) preparar la infraestructura para excluir documentos de guías cuando tengamos la lista (la generaremos en el NB03, pero aquí dejamos el hook).

**Por qué doc_uid**
- Porque si el unannotated no tiene id, necesitas una clave estable para:
- excluir documentos usados en guías,
- referenciar predicciones,
- auditar errores,
- reproducir splits.

La forma estándar: hash del texto (SHA1).

In [1]:
from pathlib import Path
ROOT = Path("/home/jovyan/inesagent")
assert ROOT.exists()
print("ROOT:", ROOT)


ROOT: /home/jovyan/inesagent


In [2]:
#Rutas y carga
from pathlib import Path
import json
import hashlib

ROOT = Path("/home/jovyan/inesagent")
PATH_UNANNOT = ROOT / "data" / "corpus_unannotated.jsonl"

OUT_SPLITS = ROOT / "outputs" / "splits"
OUT_SPLITS.mkdir(parents=True, exist_ok=True)

def load_jsonl(path):
    rows = []
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            rows.append(json.loads(line))
    return rows

unannot = load_jsonl(PATH_UNANNOT)
len(unannot)


373

In [3]:
#Creamos doc_uid determinista para los documentos que no están etiquetados (cada doc es una línea de json)
def stable_uid(text: str) -> str:
    return hashlib.sha1(text.encode("utf-8")).hexdigest()

def add_uids_unannot(rows):
    out = []
    for r in rows:
        text = r["text"]
        uid = stable_uid(text)
        out.append({"doc_uid": uid, "id": uid, "text": text})
    return out

unannot_uid = add_uids_unannot(unannot)
unannot_uid[0]["doc_uid"], unannot_uid[0]["text"][:80]


('9fd56032e0b2b4d832dc8b8f8c35ec4b9fad3074',
 'FORMALIZACIÓN DEL CONTRATO DENOMINADO “APROVECHAMIENTO DE LOS PASTOS DEL TINAJER')

Como hemos identificado, en los experimentos 2 y 3 se usan las guías como parte del prompt, con el principal **riesgo de que luego evaluemos sobre textos que el modelo ha visto literalmente en el prompt (leakage)** (nuestras guías incluyen 1 ejemplo). Por tanto, debemo garantizar que los textos usados en **memoria/few-shot no estén en prompt_regression/val/test**. Así que, para nuestro MVP, apartamos los documentos completos donde aparezcan los ejemplos de las guías. Haremos:
- Construir memoria/few-shot a partir de gold (con offsets), usando los ejemplos de guías solo como “plantilla de criterio”.
- Para evitar leakage de manera sencilla: excluir de prompt_regression/val/test los documentos que contengan cualquier cita literal usada en memoria/few-shot.

Esto es una versión híbrida:
- no excluimos “porque aparecen en la guía” en abstracto,
- excluimos solo los docs que contienen exactamente los extractos que tú vas a meter en el prompt.

Habría otra forma de hacerlo, que es, de acuerdo con el formato utilizado en las guías, los ejemplos de anotación van entre corchetes ([...]), pudiendo extraer concretamente esos trozos, pero caeríamos en el riesgo de desconfigurar los offsets. 

**_Opción A (más segura, más simple): excluir documentos que contienen citas de guía_**

- Pros: muy robusta, fácil, cero riesgo de que el modelo “vea” el mismo texto en evaluación.
- Contras: perdemos algunos documentos.

Importante: excluir documentos completos no toca offsets, porque no modificamos el texto; solo cambiamos qué docs entran en splits.

**_Opción B (más fina, conserva docs): NO excluir docs, pero excluir esos spans de cualquier “memoria/few-shot” y de tu set de evaluación_**

- Pros: no pierdes documentos.
- Contras: más compleja; si el documento completo aparece en train, y el mismo fragmento está en la guía, sigue existiendo potencial leakage “por contenido”, aunque no usemos el fragmento como ejemplo.

**Nos quedamos con la opción A.**

In [4]:
#en NB03 generaremos outputs/memory/excluded_doc_uids.json. Con estas líneas dejamos el mecanismo preparado

EXCLUDED_PATH = ROOT / "outputs" / "memory" / "excluded_doc_uids.json"

excluded = set()
if EXCLUDED_PATH.exists():
    excluded = set(json.loads(EXCLUDED_PATH.read_text(encoding="utf-8")))

unannot_filtered = [d for d in unannot_uid if d["doc_uid"] not in excluded]
len(unannot_filtered), len(unannot_uid), len(excluded)


(373, 373, 0)

**Split reproducible (train/val/test + prompt_regression)**

Recomendación para un MVP:
- test: 10%
- val: 10%
- prompt_regression: 20-40 docs (o 2-3% si el corpus es pequeño)
- train: resto

Aplicaremos:
- prompt_regression_n = 30 (ajustable)
- val_pct=0.10, test_pct=0.10

**A continuación, las celdas de splits que en [11] reescribiremos para crear el filtro anti-leakage >>**

In [5]:
import random

SEED = 42
random.seed(SEED)

docs = unannot_filtered[:]  # copia
random.shuffle(docs)

prompt_regression_n = min(30, max(10, len(docs)//50))  # regla simple adaptable
prompt_regression = docs[:prompt_regression_n]
rest = docs[prompt_regression_n:]

test_n = int(0.10 * len(rest))
val_n  = int(0.10 * len(rest))

test = rest[:test_n]
val  = rest[test_n:test_n+val_n]
train = rest[test_n+val_n:]

len(train), len(val), len(test), len(prompt_regression)


(291, 36, 36, 10)

In [6]:
#Guardamos JSONL. Con ensure_ascii=False nos aseguramos que el JSONL guardado sea legible (tildes visibles). No afecta a la carga posterior

def save_jsonl(rows, path):
    with open(path, "w", encoding="utf-8") as f:
        for r in rows:
            f.write(json.dumps(r, ensure_ascii=False) + "\n")

save_jsonl(train, OUT_SPLITS / "train.jsonl")
save_jsonl(val, OUT_SPLITS / "val.jsonl")
save_jsonl(test, OUT_SPLITS / "test.jsonl")
save_jsonl(prompt_regression, OUT_SPLITS / "prompt_regression.jsonl")


## Retomamos NB02 después de crear doc bloqueo en NB03
Reharemos NB02 usando el fichero de bloqueo.

Objetivo de NB02 ahora:
- Generar train/val/test/prompt_regression
- asegurando que NINGÚN doc_uid en blocked_doc_uids_by_memory.json aparezca en:
    - val
    - test
    - prompt_regression

(En train podemos decidir incluirlos o no; para MVP, lo habitual es permitirlos en train pero nunca en evaluación.)

In [7]:
import json
from pathlib import Path

ROOT = Path("/home/jovyan/inesagent")
# Preferimos el formato nuevo si existe
PATH_BLOCKED_KEYS = ROOT / "outputs" / "memory" / "blocked_keys_by_memory.json"
PATH_BLOCKED_LEGACY = ROOT / "outputs" / "memory" / "blocked_doc_uids_by_memory.json"

blocked = set()
if PATH_BLOCKED_KEYS.exists():
    obj = json.loads(PATH_BLOCKED_KEYS.read_text(encoding="utf-8"))
    blocked = set([str(x) for x in obj.get("blocked_ids", [])] + [str(x) for x in obj.get("blocked_uids", [])])
elif PATH_BLOCKED_LEGACY.exists():
    blocked = set(str(x) for x in json.loads(PATH_BLOCKED_LEGACY.read_text(encoding="utf-8")))

def doc_key(d: dict) -> str:
    return str(d.get("id") or d.get("doc_uid"))

# cuando tengas tu lista de docs con key estable:
docs = [d for d in docs if doc_key(d) not in blocked]


Después de esto, hacemos los splits con docs ya filtrados (solo para val/test/prompt_regression [no mantenerlos en train]
Decisión rápida (para que no haya ambigüedad)
- Para el MVP:
    - val/test/prompt_regression: excluir bloqueados
    - train: incluir bloqueados (no afecta a leakage de evaluación en prompting; pero si luego hacemos fine-tuning, lo revisamos)

- La lógica base es correcta siempre que **unannot_filtered** ya sea la lista de documentos “candidatos a split” con **doc_uid**.
- Con el output que tuvimos (291, 36, 36, 10) cuadra perfectamente con:
    - primero separas prompt_regression (10),
    - luego haces 10% test y 10% val del resto.

Dicho eso, ahora que ya tenemos **blocked_doc_uids_by_memory.json**, hay dos ajustes imprescindibles para que esté “bien” metodológicamente:

1) Asegurar anti-leakage en val/test/prompt_regression
- Necesitamos filtrar (excluir) los docs cuyo doc_uid esté en blocked antes de crear splits (o, alternativamente, permitirlos en train pero nunca en val/test/prompt_regression).

**Reemplazo recomendado (mínimo cambio)**
- Justo antes de docs = unannot_filtered añadimos la sustitución de uno por otro
- A partir de ahí, en tus splits, usa **docs_for_eval** en vez de **unannot_filtered**:
- Con esto garantizamos que **prompt_regression/val/test** no contengan documentos usados en memoria.

In [8]:
import json
from pathlib import Path

ROOT = Path("/home/jovyan/inesagent")
# Preferimos el formato nuevo si existe
PATH_BLOCKED_KEYS = ROOT / "outputs" / "memory" / "blocked_keys_by_memory.json"
PATH_BLOCKED_LEGACY = ROOT / "outputs" / "memory" / "blocked_doc_uids_by_memory.json"

blocked = set()
if PATH_BLOCKED_KEYS.exists():
    obj = json.loads(PATH_BLOCKED_KEYS.read_text(encoding="utf-8"))
    blocked = set([str(x) for x in obj.get("blocked_ids", [])] + [str(x) for x in obj.get("blocked_uids", [])])
elif PATH_BLOCKED_LEGACY.exists():
    blocked = set(str(x) for x in json.loads(PATH_BLOCKED_LEGACY.read_text(encoding="utf-8")))

def doc_key(d: dict) -> str:
    return str(d.get("id") or d.get("doc_uid"))

# Docs disponibles para evaluación (anti-leakage)
docs_for_eval = [d for d in unannot_filtered if doc_key(d) not in blocked]

#reescribimos: usaremos docs_for_eval en vez de unnannot_filtered
docs = docs_for_eval[:]  # copia
random.shuffle(docs)

prompt_regression_n = min(30, max(10, len(docs)//50))  # regla simple adaptable
prompt_regression = docs[:prompt_regression_n]
rest = docs[prompt_regression_n:]

test_n = int(0.10 * len(rest))
val_n  = int(0.10 * len(rest))

test = rest[:test_n]
val  = rest[test_n:test_n+val_n]
train = rest[test_n+val_n:]

len(train), len(val), len(test), len(prompt_regression)


(279, 34, 34, 10)

**En el anterior output tuvimos:**
(291, 36, 36, 10), **ahora tenemos** (279, 34, 34, 10). Con esto garantizamos que prompt_regression/val/test no contiene documentos usados en memoria.

In [9]:
#Y el guardado igual (nos aseguramos de tener import json arriba)

def save_jsonl(rows, path):
    with open(path, "w", encoding="utf-8") as f:
        for r in rows:
            f.write(json.dumps(r, ensure_ascii=False) + "\n")

save_jsonl(train, OUT_SPLITS / "train.jsonl")
save_jsonl(val, OUT_SPLITS / "val.jsonl")
save_jsonl(test, OUT_SPLITS / "test.jsonl")
save_jsonl(prompt_regression, OUT_SPLITS / "prompt_regression.jsonl")


## Sanity check
Después de generar splits, debe dar 0, 0, 0.

In [10]:
import json
from pathlib import Path

ROOT = Path("/home/jovyan/inesagent")
# Preferimos el formato nuevo si existe
PATH_BLOCKED_KEYS = ROOT / "outputs" / "memory" / "blocked_keys_by_memory.json"
PATH_BLOCKED_LEGACY = ROOT / "outputs" / "memory" / "blocked_doc_uids_by_memory.json"

blocked = set()
if PATH_BLOCKED_KEYS.exists():
    obj = json.loads(PATH_BLOCKED_KEYS.read_text(encoding="utf-8"))
    blocked = set([str(x) for x in obj.get("blocked_ids", [])] + [str(x) for x in obj.get("blocked_uids", [])])
elif PATH_BLOCKED_LEGACY.exists():
    blocked = set(str(x) for x in json.loads(PATH_BLOCKED_LEGACY.read_text(encoding="utf-8")))

def doc_key(d: dict) -> str:
    return str(d.get("id") or d.get("doc_uid"))

def count_blocked(rows):
    return sum(1 for r in rows if doc_key(r) in blocked)

print("blocked en prompt_regression:", count_blocked(prompt_regression))
print("blocked en val:", count_blocked(val))
print("blocked en test:", count_blocked(test))


blocked en prompt_regression: 0
blocked en val: 0
blocked en test: 0


Con esto, podemos ver:
- Estamos filtrando explícitamente docs_for_eval = [d for d in unannot_filtered if d["doc_uid"] not in blocked] antes de crear splits.
- El sanity check da: 0, 0, 0. Eso prueba que ningún documento usado en memoria/few-shot aparece en prompt_regression, val o test.

La bajada de tamaños de (291, 36, 36, 10) a (279, 34, 34, 10) es exactamente lo esperable: hemos excluido algunos documentos que estaban en unannot_filtered y que coinciden con blocked_doc_uids_by_memory.json.

A continuación, haremos dos comprobaciones adicionales (opcionales, pero buenas)
1) ¿Cuántos docs se excluyeron? (Deben ser 16, 4 de cada etiqueta)
2) Verificar que los doc_uid de memoria están efectivamente ausentes (Debe ser 0)

In [11]:
#1.¿Cuántos docs se excluyeron?
print("Total antes:", len(unannot_filtered))
print("Total eval sin leakage:", len(docs_for_eval))
print("Excluidos por memoria:", len(unannot_filtered) - len(docs_for_eval))


Total antes: 373
Total eval sin leakage: 357
Excluidos por memoria: 16


In [12]:
import json
from pathlib import Path

ROOT = Path("/home/jovyan/inesagent")
# Preferimos el formato nuevo si existe
PATH_BLOCKED_KEYS = ROOT / "outputs" / "memory" / "blocked_keys_by_memory.json"
PATH_BLOCKED_LEGACY = ROOT / "outputs" / "memory" / "blocked_doc_uids_by_memory.json"

blocked = set()
if PATH_BLOCKED_KEYS.exists():
    obj = json.loads(PATH_BLOCKED_KEYS.read_text(encoding="utf-8"))
    blocked = set([str(x) for x in obj.get("blocked_ids", [])] + [str(x) for x in obj.get("blocked_uids", [])])
elif PATH_BLOCKED_LEGACY.exists():
    blocked = set(str(x) for x in json.loads(PATH_BLOCKED_LEGACY.read_text(encoding="utf-8")))

def doc_key(d: dict) -> str:
    return str(d.get("id") or d.get("doc_uid"))

#2.Verificar que los doc_uid de memoria están efectivamente ausentes 
uids_eval = {doc_key(d) for d in docs_for_eval}
missing = [u for u in blocked if u in uids_eval]
print("Intersección blocked ∩ eval:", len(missing))


Intersección blocked ∩ eval: 0


## Qué sigue después (NB02 → NB04 / NB05)

Ya tenemos:
- splits limpios (train/val/test/prompt_regression)
- memoria final
- bloqueo anti-leakage consistente

El siguiente notebook que toca es:

**NB04 — Experimentos 1/2/3 (prompting)**
1. Cargar prompt_regression.jsonl (para depurar prompts rápido)
2. Definir los 3 prompts:
    - Exp1: solo nombres de etiquetas (sin criterios)
    - Exp2: criterios + memoria (tu memory_selected_FINAL)
    - Exp3: criterios + memoria + few-shot adicional
3. Generar predicciones JSON estricto (start/end/label) por documento
4. Guardar:
    - outputs/predictions/exp1.jsonl
    - outputs/predictions/exp2.jsonl
    - outputs/predictions/exp3.jsonl

**NB05 — Evaluación**
- Comparar pred vs gold (exact match + overlap)
- Métricas por etiqueta
- Error analysis (casos fallidos con texto y spans)